In [ ]:
!unzip MachineryNoise.zip -d MachineryNoise

In [ ]:
!unzip Nonspeech.zip -d Nonspeech

In [ ]:
!rm -rf normal_train_noise/.ipynb_checkpoints/

In [ ]:
import scipy.io.wavfile as wav
import os
import librosa

In [ ]:
# each is 30 sec, too long so 10 sec, 10 sec
# discard first and last 5 sec
path = 'MachineryNoise/MachineryNoise/'
files = os.listdir(path)
out_path = 'normal_train_noise/'
for f in files:
  # convert 44khz to 16khz
  audio, fs = librosa.load(path+f, sr=16000, mono=True)
  l = len(audio)
  # about 4, 5, 6, 8, 4, 3 sec
  lengths = [0, l//8, l//6, l//5, l//4, l//8]
  lengths.append(len(audio)-sum(lengths))
  for i in range(1, len(lengths)):
    lengths[i] += lengths[i-1]
  # cut off .wav
  new_f = f[:-4]
  for i in range(1, len(lengths)):
    wav.write(out_path+new_f+f'_{i-1}.wav', fs, audio[lengths[i-1]:lengths[i]]) 

In [ ]:
# convert nonspeech from 20khz to 16khz
path = 'normal_train_noise/'
files = [i for i in os.listdir(path) if len(i) < 8]
assert len(files) == 99
for f in files:
  audio, fs = librosa.load(path+f, sr=16000, mono=True)
  wav.write(out_path+f, fs, audio)

In [ ]:
mach = [i for i in os.listdir('normal_train_noise') if '_5.wav' in i]
# 694-675 = 19, get rid of 
for i in range(19):
  os.popen(f'rm {out_path+mach[i]}')

In [ ]:
print(len(os.listdir(out_path)))
assert len(os.listdir(out_path)) == 675

675


In [ ]:
!rm -rf set/train_noise/

!cp -r normal_train_noise/ set/train_noise

In [ ]:
fs, _ = wav.read('/content/set/train_noise/Machinery101_0.wav')
assert fs == 16000
fs, _ = wav.read('/content/set/train_clean_speech/001364.wav')
assert fs == 16000
fs, _ = wav.read('/content/set/test_noisy_speech/adv-long2long-000003.wav')
assert fs == 16000
fs, _ = wav.read('/content/set/test_clean_speech/000003.wav')
assert fs == 16000

In [ ]:
!tar -czf test_train_split_normal.tar.gz set/